In [ ]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
from operator import truediv

In [ ]:
#compute PageRank algorith with power iteration
def pageRank(M, t, iterations):
    N = M.shape[1]
    c = 0.85
    p = np.ones(N)/N

    for i in range(iterations):
        p = c * M * p + (1-c) * t #pagerank formula
        p = p/sum(p) #normalization

    return p

In [ ]:
path = 'data/wiki-Vote.txt'
G = nx.read_edgelist(path, delimiter='\t', create_using=nx.DiGraph, nodetype=int)
A = nx.linalg.graphmatrix.adjacency_matrix(G)
N = A.shape[0]
print(nx.info(G))

In [ ]:
plt.spy(A, marker=',')
plt.show()

In [ ]:
#for normal pagerank all nodes have the same teleport probability
#t = np.ones(N)/N 

#for local pagerank only a subset of nodes have non-negative probabilty
t = np.zeros(N)
t[3000] = 1

p = pageRank(A, t, 40)
plt.plot(p)
plt.show()

FIRST TRIAL





In [ ]:
def push(node, pr_vector, residuals, G, degrees):
    
    # constant
    alfa = 0.15
    # actual residual of the target node
    current_res = residuals[str(node)]

    # compute the new rank for the target node
    pr_vector[str(node)] += alfa * current_res
    # decrease the residual of the target node
    residuals[str(node)] = (1-alfa) * current_res / 2

    # calculate the residuals for the neighbors of the target node
    for neighbour in G.out_degree(node):
        residuals[str(neighbour)] += (1-alfa) * current_res / (2 * degrees[str(node)])

    return pr_vector, residuals

In [ ]:
def pageRankApproximate(G, epsilon, node):

    all_nodes = G.nodes()

    # extract the dimension of the matrix
    N = len(all_nodes)
    # degrees of the nodes
    degrees = {}
    for n in all_nodes:
        degrees[str(n)] = G.degree(n)

    # pagerank vector
    pr_vector = {}
    for i in all_nodes:
        pr_vector[str(i)] = 0

    # residuals vector
    residuals = {}
    for i in all_nodes:
        residuals[str(i)] = 0
    residuals[str(node)] = 1

    # residuals/nodes' degree
    res_over_deg = {}
    for i in all_nodes:
        val = residuals[str(i)] / degrees[str(i)]
        if (val > 0):
            res_over_deg[str(i)] = val

    while (bool(res_over_deg)):
        queue = list(res_over_deg.keys())
        while (len(queue) > 0):
            u = queue[0]
            while (res_over_deg[u] >= epsilon):
                pr_vector, residuals = push(u, pr_vector, residuals, G, degrees)
                res_over_deg.clear()
                for i in all_nodes:
                    val = residuals[str(i)] / degrees[str(i)]
                    if (val >= epsilon):
                        res_over_deg[str(i)] = val
            del queue[0]
        
            
        res_over_deg.clear()
        for i in all_nodes:
            val = residuals[str(i)] / degrees[str(i)]
            if (val > 0):
                res_over_deg[str(i)] = val

    return pr_vector

In [ ]:
p = pageRankApproximate(G, 0.01, 3000)
plt.plot(p)
plt.show()

SECOND TRIAL (fast approach)

In [ ]:
def fastPRApproximate(M, epsilon, node):

    # extract the dimension of the matrix
    N = M.shape[1]
    # degrees of the nodes
    degrees = sp.sparse.csr_matrix.sum(M, axis=1)
    degrees = np.squeeze(np.asarray(degrees))

    alpha = 0.85

    # pagerank vector
    pr_vector = np.zeros(N)

    # residuals vector
    residuals = np.zeros(N)
    residuals[node] = 1

    # residuals/nodes' degree
    res_over_deg = list(map(truediv, residuals, degrees))

    while (not isEmpty(find(res_over_deg, alpha*epsilon))):
        queue = find(res_over_deg, alpha*epsilon)
        while (not isEmpty(queue)):
            v = queue[0]
            pr_vector[v] += residuals[v]
            m = (1 - alpha) * residuals[v] / degrees[v]
            residuals[v] = 0
            for u in connectedNodes(v, M):
                residuals[u] += m
            del queue[0]
        res_over_deg = list(map(truediv, residuals, degrees))

    return pr_vector

In [ ]:
p = fastPRApproximate(A, 0.01, 3000)
plt.plot(p)
plt.show()

THIRD TRIAL

In [ ]:
def ghesboro(M, alpha, epsilon, node):

    alpha_eps = alpha*epsilon

    # extract the dimension of the matrix
    N = M.shape[1]

    # degrees of the nodes
    degrees = sp.sparse.csr_matrix.sum(M, axis=1)
    degrees = np.squeeze(np.asarray(degrees))

    # pagerank vector
    pr_vector = np.zeros(N)

    # residuals vector
    residuals = np.zeros(N)
    residuals[node] = alpha

    queue = [node]

    while (len(queue) > 0):

        u = queue[0]
        res = residuals[u]

        pr_vector += res
        residuals[u] = 0

        for v in connectedNodes(u, M):

            val = (1 - alpha) * res / degrees[u]
            residuals[v] += val

            res_v = residuals[v]
            if res_v >= alpha_eps * degrees[v]:
                queue.append(v)

        del queue[0]
    
    return pr_vector

In [ ]:
p = ghesboro(A, 0.85, 0.01, 3000)
plt.plot(p)
plt.show()